In [1]:
import os

import pandas as pd
from pandas import DataFrame
import numpy as np
from scipy.stats.mstats import winsorize

from Constants import Constants as const

In [2]:
pc3_df: DataFrame = pd.read_stata(os.path.join(const.DATA_PATH, 'fromZGY', 'firm_year_num_political_donar_3years.dta')).rename(
    columns={'recipient_ext_id': 'numDonation3year'}).drop(['index'], axis=1)
pc4_df: DataFrame = pd.read_stata(os.path.join(const.DATA_PATH, 'fromZGY', 'firm_year_num_political_donar_4year.dta')).rename(
    columns={'recipient_ext_id': 'numDonation4year'}).drop(['index'], axis=1)

In [7]:
pc3_avg = pc3_df.groupby([const.GVKEY])['numDonation3year'].mean()
pc4_avg = pc4_df.groupby([const.GVKEY])['numDonation4year'].mean()

In [9]:
pc3_avg_df: DataFrame = pc3_avg.reset_index(drop=False)
pc4_avg_df: DataFrame = pc4_avg.reset_index(drop=False)

In [10]:
reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20241014_stock_act_reg_data_v2.dta'))
gvkey_series = reg_df[const.GVKEY].unique()

C:\Users\wangy\AppData\Local\Temp\ipykernel_20864\88727916.py:1: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20241014_stock_act_reg_data_v2.dta'))


In [13]:
# Update the function to handle numpy.ndarray as input
def ensure_gvkeys_with_ndarray(df, gvkey_array, num_col_name):
    # Convert the numpy array to a set for quick lookup
    gvkey_set = set(gvkey_array)
    # Identify missing gvkeys
    existing_gvkeys = set(df['gvkey'])
    missing_gvkeys = gvkey_set - existing_gvkeys
    # Create a DataFrame with missing gvkeys
    missing_rows = pd.DataFrame({
        'gvkey': list(missing_gvkeys),
        num_col_name: [None] * len(missing_gvkeys)
    })
    # Concatenate the original DataFrame with missing rows and sort by gvkey
    updated_df = pd.concat([df, missing_rows], ignore_index=True)
    updated_df = updated_df.sort_values('gvkey').reset_index(drop=True)
    return updated_df

In [14]:
# Update pc3_avg_df and pc4_avg_df
pc3_avg_df2 = ensure_gvkeys_with_ndarray(pc3_avg_df, gvkey_series, 'numDonation3year')
pc4_avg_df2 = ensure_gvkeys_with_ndarray(pc4_avg_df, gvkey_series, 'numDonation4year')


C:\Users\wangy\AppData\Local\Temp\ipykernel_20864\1236285319.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_df = pd.concat([df, missing_rows], ignore_index=True)


In [16]:
pc_avg_df: DataFrame = pc3_avg_df2.merge(pc4_avg_df2, on=[const.GVKEY])
pc_avg_df.shape

(1837, 3)

In [17]:
for key in ['numDonation3year', 'numDonation4year']:
    pc_avg_df[key] = pc_avg_df[key].fillna(0)

In [20]:
pc_avg_df['highDonation3Year'] = (pc_avg_df['numDonation3year'] > pc_avg_df['numDonation3year'].median()).astype(int)
pc_avg_df['highDonation4Year'] = (pc_avg_df['numDonation4year'] > pc_avg_df['numDonation4year'].median()).astype(int)

In [22]:
reg_df2: DataFrame = reg_df.merge(pc_avg_df, on=[const.GVKEY], how='left')
reg_df2.to_stata(os.path.join(const.RESULT_PATH, '20241212_stock_act_reg_data.dta'), write_index=False, version=119)